In [1]:
# import the necessary packages
import pandas as pd
import pickle
import datetime
from dateutil import parser
import re

In [137]:
# data = pd.read_pickle('pickled_data.p')

text_raw = [t for t in data['description']]

#to lowercase
text_lower = [s.lower() for s in text_raw]

#remove any of below characters and replace with space - excesses will be removed later
text_lower = [re.sub('[#!?*";,())]',' ',s) for s in text_lower]

#remove text artifacts
text_lower = [re.sub('website_redacted',' ',s) for s in text_lower]
text_lower = [re.sub('\xa0',' ',s) for s in text_lower]
text_lower = [re.sub('\r','',s) for s in text_lower]
text_lower = [re.sub('\t','',s) for s in text_lower]
text_lower = [re.sub('_',' ',s) for s in text_lower]

# remove numbers of 3 or more digits as long as they are not preceded by a $
# text_lower = [re.sub('(?<!([$]|\d))\d{3,}',' ',s) for s in text_lower]

#remove $ only if it does not precede a number then remove all numbers
text_lower = [re.sub('([$](?!\d))',' ',s) for s in text_lower]
text_lower = [re.sub('\d{3,}',' ',s) for s in text_lower]
text_lower = [re.sub('.00',' ',s) for s in text_lower]

#remove below characters only if they are not preceded by a number
text_lower = [re.sub('((?<!\d)[.:-])',' ',s) for s in text_lower]

#insert spaces between letters and numbers where 
text_lower = [re.sub(r'((?<=[a-z])\d)',r' \1',s) for s in text_lower]
text_lower = [re.sub(r'(\d(?=[a-z]))',r'\1 ',s) for s in text_lower]

#remove all html tags
text_lower = [re.sub('(<.*?>)',' ',s) for s in text_lower]

#replace ampersand with and
text_lower = [re.sub('[&]',' and ',s) for s in text_lower]

#replace w/ with "with"
text_lower = [re.sub('w/',' with ',s) for s in text_lower]

#replace / symbols with space
text_lower = [re.sub('/',' ',s) for s in text_lower]

#replace excess spaces
text_lower = [re.sub(' +',' ',s).strip() for s in text_lower]

#adjust important abbreviations
text_lower = [re.sub(' br ',' bedroom ',s) for s in text_lower]
text_lower = [re.sub('sq ft','square feet',s) for s in text_lower]
text_lower = [re.sub(' ss ',' stainless steel',s) for s in text_lower]
text_lower = [re.sub(' s s ',' staineless steel ',s) for s in text_lower]


#remove trailing "<a"
text_cleaned = [re.sub('<a','',s).strip() for s in text_lower]


In [139]:
data['cleantext'] = text_cleaned
data.drop('description',axis=1)
print('done')

done


In [140]:
X = data.drop(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'street_address', 'features', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'manager_id', 'photos'],axis=1)
Y = data['interest_level'].copy()

In [5]:
# Y[(Y=='low')]=1
# Y[(Y=='medium')]=2
# Y[(Y=='high')]=3
# Y.head()

In [141]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction import DictVectorizer
import numpy as np

In [142]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8)

In [187]:
# binary_vectorizer = CountVectorizer(ngram_range=(2,2),binary=True,stop_words='english')
binary_vectorizer = TfidfVectorizer(ngram_range=(1,2),stop_words='english')
binary_vectorizer.fit(X['cleantext'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [138]:
# print(re.sub(r'(\d(?=[a-z]))',r'\1 ',text_cleaned[32756]))
# m = re.search('00available',text_cleaned[32756])
# text_raw[32756]
# text_cleaned[8]
# tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_vectorizer.fit(X_train['cleantext'])

''

In [188]:
X_train_vect = binary_vectorizer.transform(X_train['cleantext'])
X_test_vect = binary_vectorizer.transform(X_test['cleantext'])

In [189]:
X_train_bin = X_train.drop(['cleantext'],axis=1)
X_train_bin['textvector'] = X_train_vect

In [ ]:
# train_tfidf_vector = tfidf_vectorizer.transform(X_train['cleantext'])
# test_tfidf_vector = tfidf_vectorizer.transform(X_test['cleantext'])

In [ ]:
# X_train_tfidf = X_train
# X_train_tfidf['textvector'] = train_tfidf_vector
# X_train_tfidf.drop('cleantext',axis=1)
# print('done')

In [ ]:
# l_model_tfidf = LogisticRegression()
# l_model_tfidf.fit(X_train_tfidf, Y_train)

In [16]:
# from sklearn.model_selection import GridSearchCV

# lr_param = {'C':[10**-i for i in range(-5,5)]}
# lr_gscv = GridSearchCV(LogisticRegression(),lr_param,cv=10)
# lr_gscv.fit(X_train_vect,Y_train)

# print(str(lr_gscv.best_params_['C']))

In [190]:
l_model_binary = LogisticRegression()
l_model_binary.fit(X_train_vect, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [191]:
test_predict = l_model_binary.predict(X_test_vect)

In [192]:
1-sum((test_predict!=Y_test)*1)/len(Y_test)


0.70489312126430959